## Background

This information has been deleted by the IronHacks Team

## Setup

In [1]:
%logstop
%logstart -t -r -q ipython_command_log.py global

#- IRONHACKS RESEARCH TRACKING CODE
#----------------------------------
# The following code is used to help our research team understand how you 
# our notebook environment. We do not collect any personal information with
# the following code, it is used to measure when and how often you work on
# your submission files.

import os
from datetime import datetime
import IPython.core.history as history

ha = history.HistoryAccessor()
ha_tail = ha.get_tail(1)
ha_cmd = next(ha_tail)
session_id = str(ha_cmd[0])
command_id = str(ha_cmd[1])
timestamp = datetime.utcnow().isoformat()
history_line = ','.join([session_id, command_id, timestamp]) + '\n'
logfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')
logfile.write(history_line)
logfile.close()


Logging hadn't been started.


In [2]:
#- INSTALL ADDITIONAL LIBRARIES IF REQUIRED
#------------------------------------------
# This is normally not required. The hub environment comes preinstaled with 
# many packages that you can already use without setup. In case there is some
# other library you would like to use that isn't on the list you run this command
# once to install them.  If it is already installed this command has no effect.

## Imports

In [4]:
#- IMPORT THE LIBRARIES YOU WILL USE
#------------------------------------------
# You only need to import packages one time per notebook session. To keep your
# notebook clean and organized you can handle all imports at the top of your file.
# The following are included for example purposed, feel free to modify or delete 
# anything in this section.


import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics

import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers


## Get data from Big Query 

### Configure the Big Query Settings

In [48]:
import os
BIGQUERY_PROJECT = 'cloud-test-ironhacks'
BIGQUERY_KEYPATH = 'cloud-test-ironhacks-052e1ced8b54.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = BIGQUERY_KEYPATH
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

## SQL Code

In [49]:
query = """
SELECT uu_id, week_number, total_claims

FROM `cloud-test-ironhacks.dwd_data.suppression_remove`

ORDER BY uu_id, week_number;
""" 

query_job = bigquery_client.query(query)
in_test_data = query_job.to_dataframe()
in_test_data.head()


,uu_id,week_number,total_claims
0,001cd9ae23064d7f0fd3cd327c873d8d,1,NaN
1,001cd9ae23064d7f0fd3cd327c873d8d,2,25.0
2,001cd9ae23064d7f0fd3cd327c873d8d,3,14.0
3,001cd9ae23064d7f0fd3cd327c873d8d,4,13.0
4,001cd9ae23064d7f0fd3cd327c873d8d,5,12.0


In [50]:
query = """
SELECT *

FROM `cloud-test-ironhacks.dwd_data.wage_uuid`


""" 

query_job = bigquery_client.query(query)
wage_data = query_job.to_dataframe()
wage_data.head()


,uu_id,countyfips,tract,tract_name,average_wage
0,9bee42b55f891413ae7fac2d9d89280a,18003,100,"Census Tract 1, Allen County, Indiana",9199.222222
1,3952d1e916f8cad4225e4b9e29f62a06,18039,100,"Census Tract 1, Elkhart County, Indiana",5586.750000
2,6ac331ecae4b7e4dd2750893612331f9,18053,100,"Census Tract 1, Grant County, Indiana",10168.470588
3,6a4357ec916e146c5329ac2498a84f66,18105,100,"Census Tract 1, Monroe County, Indiana",7601.500000
4,16b9d35c6d253c30527eb2de8d2d1bb9,18141,100,"Census Tract 1, St. Joseph County, Indiana",6278.000000


In [51]:
in_test_data['average_wage'] = in_test_data['uu_id'].map(wage_data.set_index('uu_id')['average_wage'])
in_test_data

,uu_id,week_number,total_claims,average_wage
0,001cd9ae23064d7f0fd3cd327c873d8d,1,NaN,8228.8
1,001cd9ae23064d7f0fd3cd327c873d8d,2,25.0,8228.8
2,001cd9ae23064d7f0fd3cd327c873d8d,3,14.0,8228.8
3,001cd9ae23064d7f0fd3cd327c873d8d,4,13.0,8228.8
4,001cd9ae23064d7f0fd3cd327c873d8d,5,12.0,8228.8
...,...,...,...,...
55877,ffbc87dc4bde6828daff6ad43e12db4a,33,10.0,11517.1
55878,ffbc87dc4bde6828daff6ad43e12db4a,34,NaN,11517.1
55879,ffbc87dc4bde6828daff6ad43e12db4a,35,24.0,11517.1
55880,ffbc87dc4bde6828daff6ad43e12db4a,36,22.0,11517.1


In [52]:
in_test_data = in_test_data.fillna(0)
in_test_data

,uu_id,week_number,total_claims,average_wage
0,001cd9ae23064d7f0fd3cd327c873d8d,1,0.0,8228.8
1,001cd9ae23064d7f0fd3cd327c873d8d,2,25.0,8228.8
2,001cd9ae23064d7f0fd3cd327c873d8d,3,14.0,8228.8
3,001cd9ae23064d7f0fd3cd327c873d8d,4,13.0,8228.8
4,001cd9ae23064d7f0fd3cd327c873d8d,5,12.0,8228.8
...,...,...,...,...
55877,ffbc87dc4bde6828daff6ad43e12db4a,33,10.0,11517.1
55878,ffbc87dc4bde6828daff6ad43e12db4a,34,0.0,11517.1
55879,ffbc87dc4bde6828daff6ad43e12db4a,35,24.0,11517.1
55880,ffbc87dc4bde6828daff6ad43e12db4a,36,22.0,11517.1


In [53]:
in_test_data.dtypes

uu_id            object
week_number       int64
total_claims    float64
average_wage    float64
dtype: object

In [54]:
checkpoint1 = in_test_data.copy()

In [55]:
checkpoint1['uu_id'] = pd.Categorical(checkpoint1['uu_id'])
checkpoint1['uu_id'] = checkpoint1.uu_id.cat.codes

checkpoint1.head()

,uu_id,week_number,total_claims,average_wage
0,0,1,0.0,8228.8
1,0,2,25.0,8228.8
2,0,3,14.0,8228.8
3,0,4,13.0,8228.8
4,0,5,12.0,8228.8


In [56]:
target = checkpoint1.pop('total_claims')
dataset = tf.data.Dataset.from_tensor_slices((checkpoint1.values, target.values))


In [57]:
np.savez('data_train', inputs=checkpoint1, targets=target)

In [58]:
npz = np.load('data_train.npz')

In [59]:
train_inputs = npz['inputs'].astype(np.float)

<ipython-input-59-9cc92c3e790f>:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_inputs = npz['inputs'].astype(np.float)


In [60]:
train_targets = npz['targets'].astype(np.int)

<ipython-input-60-9061bcfb1d91>:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_targets = npz['targets'].astype(np.int)


In [61]:
input_size = 3
output_size = 1
hidden_layer_size = 500

def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
        tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
        tf.keras.layers.Dense(output_size)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

In [62]:
batch_size = 43

# set a maximum number of training epochs
max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)


In [63]:
model = get_compiled_model()
model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          # callbacks are functions called by a task when a task is completed
          # task here is to check if val_loss is increasing
          #callbacks=[early_stopping], # early stopping
          #validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2 # making sure we get enough information about the training process
          )


Epoch 1/100
1300/1300 - 1s - loss: 12520.4482 - accuracy: 0.0165
Epoch 2/100
1300/1300 - 1s - loss: 1986.8298 - accuracy: 0.0242
Epoch 3/100
1300/1300 - 1s - loss: 1800.2423 - accuracy: 0.0224
Epoch 4/100
1300/1300 - 1s - loss: 1418.8143 - accuracy: 0.0135
Epoch 5/100
1300/1300 - 1s - loss: 1251.1515 - accuracy: 0.0081
Epoch 6/100
1300/1300 - 1s - loss: 1146.9622 - accuracy: 0.0062
Epoch 7/100
1300/1300 - 1s - loss: 1103.5643 - accuracy: 0.0051
Epoch 8/100
1300/1300 - 1s - loss: 1054.2122 - accuracy: 0.0045
Epoch 9/100
1300/1300 - 1s - loss: 1017.5020 - accuracy: 0.0046
Epoch 10/100
1300/1300 - 1s - loss: 966.2355 - accuracy: 0.0042
Epoch 11/100
1300/1300 - 1s - loss: 901.0092 - accuracy: 0.0042
Epoch 12/100
1300/1300 - 1s - loss: 867.2767 - accuracy: 0.0041
Epoch 13/100
1300/1300 - 1s - loss: 856.6597 - accuracy: 0.0043
Epoch 14/100
1300/1300 - 1s - loss: 853.6094 - accuracy: 0.0037
Epoch 15/100
1300/1300 - 1s - loss: 846.4548 - accuracy: 0.0038
Epoch 16/100
1300/1300 - 1s - loss: 845

In [64]:
temp = np.array([i for i in range(0, 55882)])

In [65]:
temp1 = np.array([37 for i in range(0, 55882)])

In [66]:
temp2 = np.array([1 for i in range(0, 55882)])

In [67]:
predict_data = np.stack((temp, temp1, temp2))

In [68]:
predict_data = predict_data.transpose()

In [69]:
print(predict_data)

[[    0    37     1]
 [    1    37     1]
 [    2    37     1]
 ...
 [55879    37     1]
 [55880    37     1]
 [55881    37     1]]


In [70]:
result = model.predict(predict_data)

In [71]:
print(result)

[[ -9.878493]
 [ -9.592389]
 [ -9.308654]
 ...
 [171.63011 ]
 [171.6331  ]
 [171.63611 ]]
